In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import h5py
file = h5py.File('drive/My Drive/New Data (test)/flower-river.h5', 'r')
list(file.values())

In [ ]:
pip install tensorflow-io

In [ ]:
import h5py
import numpy as np
import tensorflow_io as tfio

# Load HDF5 dataset
h5f_train_X = tfio.IODataset.from_hdf5('drive/My Drive/New Data (test)/train.h5', '/X')
h5f_train_Y = tfio.IODataset.from_hdf5('drive/My Drive/New Data (test)/train.h5', '/Y')


h5f_test_X=tfio.IODataset.from_hdf5('drive/My Drive/New Data (test)/test.h5', '/X')
h5f_test_Y=tfio.IODataset.from_hdf5('drive/My Drive/New Data (test)/test.h5', '/Y')

h5f_val_X=tfio.IODataset.from_hdf5('drive/My Drive/New Data (test)/val.h5', '/X')
h5f_val_Y=tfio.IODataset.from_hdf5('drive/My Drive/New Data (test)/val.h5', '/Y')


h5f_val_fr_X=tfio.IODataset.from_hdf5('drive/My Drive/New Data (test)/flower-river.h5', '/X')
h5f_val_fr_Y=tfio.IODataset.from_hdf5('drive/My Drive/New Data (test)/flower-river.h5', '/Y')


In [ ]:
import tensorflow as tf
# Zip together samples and corresponding labels
train = tf.data.Dataset.zip((h5f_train_X ,h5f_train_Y)).batch(5187, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
test = tf.data.Dataset.zip((h5f_test_X,h5f_test_Y)).batch(1622, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
val = tf.data.Dataset.zip((h5f_val_X,h5f_val_Y)).batch(1297, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
fr_autoEnc= tf.data.Dataset.zip((h5f_val_fr_X,h5f_val_fr_X)).batch(400, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
fr= tf.data.Dataset.zip((h5f_val_fr_X,h5f_val_fr_Y)).batch(400, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
train

In [ ]:
import keras
from matplotlib import pyplot as plt
import numpy as np
from keras.layers import Input,Dense,Flatten,Dropout,merge,Reshape,Conv2D,MaxPooling2D,UpSampling2D,Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization
from keras.models import Model,Sequential

batch_size = 128
epochs = 200
inChannel = 1
x, y = 50,50
input_img = Input(shape = (x, y, inChannel))


In [ ]:
def encoder(input_img):
    #encoder
    conv1 = Conv2D(32, (5, 5), activation='relu', padding='same')(input_img) #28 x 28 x 100
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) #14 x 14 x 100
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1) #14 x 14 x 150
    pool2 = MaxPooling2D(pool_size=(5, 5))(conv2) #7 x 7 x 150
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2) #7 x 7 x 200 (small and thick)
    return conv3
  
def decoder(conv3):    
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3) #7 x 7 x 200  
    up1 = UpSampling2D((5,5))(conv4) #14 x 14 x 150
    conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1) #7 x 7 x 150
    up2 = UpSampling2D((2,2))(conv5) # 28 x 28 x 100
    conv6 = Conv2D(32, (5, 5), activation='relu', padding='same')(up2) # 14 x 14 x 100
    decoded = Conv2D(1, (5, 5), activation='relu', padding='same')(conv6) # 28 x 28 x 1
    return decoded

def fc(encoded):
    flat=Flatten()(encoded)
    dp = Dropout(0.4, seed=42)(flat)
    den1 = Dense(256, activation='relu')(dp)
    den2=Dense(2, activation='softmax')(den1)
    return den2


In [ ]:
autoencoder=Model(input_img, decoder(encoder(input_img)))
autoencoder.compile(loss='mean_squared_error', optimizer = 'adam')
autoencoder.summary()

In [ ]:
aut_train=autoencoder.fit(fr_autoEnc, batch_size=128 ,epochs=200, shuffle="batch")

In [ ]:
import matplotlib.pyplot as plt
decoded_imgs = autoencoder.predict(X_test_images[100:])

n = 10
plt.figure(figsize=(20, 4))
for i in range(10):
    # display original
    ax = plt.subplot(2, n, i+1)
    if np.array_equal(Y_f_labels[i+110],[1., 0.]):
      ax.set_title("River")
    else:
      ax.set_title("Flower")
    plt.imshow(X_f_images[i+110].reshape(50, 50))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + n+1)
    plt.imshow(decoded_imgs[i].reshape(50, 50))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
encode=encoder(input_img)
model = Model(input_img, fc(encode))
for l1,l2 in zip(model.layers[0:6],autoencoder.layers[0:6]):
    l1.set_weights(l2.get_weights())


In [ ]:
model.compile(loss='mean_squared_error', optimizer=opt ,metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(fr,
          batch_size=128,
          epochs=200,
          shuffle='batch')

In [ ]:
history=model.fit(train,
          batch_size=128,
          epochs=200,
          shuffle='batch',
          validation_data=(val),class_weight={0:1, 1:6})

In [ ]:
from keras.models import load_model
score = model.evaluate(test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
y_pred = np.argmax(Y_test_labels, axis=1)
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

predictions_friv=model.predict(X_test_images)
matrix_friv = confusion_matrix(y_pred, predictions_friv.argmax(axis=1))
sns.heatmap(matrix_friv, cmap="Blues",annot=True, fmt=".0f")


In [ ]:
print(classification_report(y_pred, predictions_friv.argmax(axis=1)))